# Import libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy import sparse

# Download data

In [2]:
!wget https://snap.stanford.edu/data/loc-gowalla_edges.txt.gz
!wget https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz

!gzip -vd loc-gowalla_edges.txt.gz
!gzip -vd loc-gowalla_totalCheckins.txt.gz

--2021-11-03 17:48:49--  https://snap.stanford.edu/data/loc-gowalla_edges.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6351523 (6.1M) [application/x-gzip]
Saving to: ‘loc-gowalla_edges.txt.gz’

loc-gowalla_edges.t 100%[===================>]   6.06M  2.48MB/s    in 2.4s    

2021-11-03 17:48:51 (2.48 MB/s) - ‘loc-gowalla_edges.txt.gz’ saved [6351523/6351523]

--2021-11-03 17:48:51--  https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105470044 (101M) [application/x-gzip]
Saving to: ‘loc-gowalla_totalCheckins.txt.gz’

loc-gowalla_totalCh 100%[===================>] 100.58M  17.4MB/s    in 5.2s    

2021-11-03 17:48

In [3]:
df = pd.read_csv(
    filepath_or_buffer="loc-gowalla_edges.txt",
    sep="\t",
    header=None,
    names=["from", "to"],
)

In [4]:
%%capture --no-stdout

N = 1000

row_ind = df["from"].values
row_ind = row_ind[row_ind < N]

col_ind = df["to"].values
col_ind = col_ind[col_ind < N]

data = np.ones(row_ind.shape[0], dtype=np.float64)

# initialize similarities
S = sparse.csr_matrix((data, (row_ind, col_ind)), shape=(N, N))
S.setdiag(-1.0)

# Affinity Propagation


In [5]:
%%capture --no-stdout

data = np.zeros(row_ind.shape[0], dtype=np.float32)

# initialize availability
A = sparse.csr_matrix((data, (row_ind, col_ind)), shape=(N, N))

# initialize responsibility
R = sparse.csr_matrix((data, (row_ind, col_ind)), shape=(N, N))

max_iterations = 42

for iteration in range(max_iterations):
    R_old = R.copy()
    A_old = A.copy()

    # update responsibility
    sum = S + A

    first_max_indices = np.asarray(np.argmax(sum, axis=-1)).flatten()
    first_max = np.asarray(sum[np.arange(N), first_max_indices]).flatten()

    tmp = sum.copy()
    tmp[np.arange(N), first_max_indices] = -np.inf

    second_max_indices = np.asarray(np.argmax(tmp, -1)).flatten()
    second_max = np.asarray(sum[np.arange(N), second_max_indices]).flatten()

    for r_ind, (i, j) in enumerate(zip(R.indptr, R.indptr[1:])):
        R.data[i:j] = S.data[i:j] - first_max[r_ind]
        if first_max_indices[r_ind] in R.indices[i:j]:
            R[r_ind, first_max_indices[r_ind]] = S[r_ind, first_max_indices[r_ind]] - second_max[r_ind]

    # update availability
    A = R.copy()
    A.setdiag(0)
    A[A < 0] = 0

    sums = np.asarray(np.sum(A, axis=0)).flatten()
    sums_d = R.diagonal() + sums

    A.data = np.minimum(0, sums_d[A.indices] - A.data)
    A.setdiag(sums)

    result = A + R
    labels = [np.argmax(result[i]) for i in range(N)]

    A_loss = np.abs(A - A_old).sum()
    R_loss = np.abs(R - R_old).sum()

    print(f"Iteration #{iteration + 1}")
    print(f"  Availability loss: {A_loss}")
    print(f"  Responsibility loss: {R_loss}")
    print(f"  Number of clusters: {np.unique(np.array(labels)).shape[0]}")

    if A_loss == 0.0 and R_loss == 0.0:
        break

Iteration #1
  Availability loss: 498603.0
  Responsibility loss: 4108987.0
  Number of clusters: 267
Iteration #2
  Availability loss: 1157520.0
  Responsibility loss: 12861393.0
  Number of clusters: 269
Iteration #3
  Availability loss: 210550.0
  Responsibility loss: 4656784.0
  Number of clusters: 263
Iteration #4
  Availability loss: 15141.0
  Responsibility loss: 1966137.0
  Number of clusters: 263
Iteration #5
  Availability loss: 5457.0
  Responsibility loss: 352661.0
  Number of clusters: 263
Iteration #6
  Availability loss: 30.0
  Responsibility loss: 21816.0
  Number of clusters: 263
Iteration #7
  Availability loss: 14.0
  Responsibility loss: 1197.0
  Number of clusters: 263
Iteration #8
  Availability loss: 4.0
  Responsibility loss: 849.0
  Number of clusters: 263
Iteration #9
  Availability loss: 0.0
  Responsibility loss: 376.0
  Number of clusters: 263
Iteration #10
  Availability loss: 0.0
  Responsibility loss: 0.0
  Number of clusters: 263


# Predict checkins

In [6]:
from collections import Counter

checkins = pd.read_csv(
    filepath_or_buffer='loc-gowalla_totalCheckins.txt',
    delimiter = '\t',
    names = ['user', 'time', 'latitude', 'longitude', 'location_id'],
)

users = checkins['user'].unique()
users = users[users < N]

np.random.shuffle(users)

test_users = users[:len(users) // 10]

clusters = defaultdict(list)
for user, i in enumerate(labels):
    clusters[i].append(user)

loc_counter = defaultdict(Counter)
for i, vals in clusters.items():
    data = checkins[checkins['user'].isin(vals)]['location_id'].values
    loc_counter[i].update(data)

In [7]:
accuracies = []
for user in test_users:
    i = labels[user]
    accuracy = len(
        set([location_id for location_id, _ in loc_counter[i].most_common(10)])
        & set(checkins[checkins['user'] == user]['location_id'].values)
    ) / 10
    accuracies.append(accuracy)
print(f"accuracy: {np.mean(accuracies)}")

accuracy: 0.28620689655172415
